# BAC0

BAC0 is a script module based on bacpypes (https://github.com/JoelBender/bacpypes.git) to automate task using Bacnet/IP protocole. It's mainly targetted as a tool to test sequences on DDC controlelrs using bacnet.

Using the simplciity of python, the power of pandas, it's easy to test and keep a trace of the history of each points.

## How to use

First you must install the 2 modules
pip install bacpypes
- git clone https://github.com/ChristianTremblay/BAC0.git (python setup.py install)

Note : All tests have been done in Windows 7 using Anaconda3 (http://continuum.io/downloads)

## Import module

In [2]:
import BAC0
import pandas as pd
%matplotlib inline
# Define network and connect to it
bacnet = BAC0.connect(ip='192.168.56.201/24',mask='255.255.255.0')
#bacnet_2 = BAC0.connect(ip='192.168.56.203/24',mask='255.255.255.0', port=47810)

2020-10-01 12:17:27,222 - INFO    | Starting BAC0 version 20.02.20 (Lite)
2020-10-01 12:17:27,223 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2020-10-01 12:17:27,224 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2020-10-01 12:17:27,226 - INFO    | Using ip : 192.168.56.201


InitializationError: IP Address provided (192.168.56.201) already used by BAC0. Check if another software is using port 47808 on this network interface. If so, you can define multiple IP per interface. Or specify another IP using BAC0.lite(ip='IP/mask')

In [143]:
BAC0.version

'20.02.20'

### Finding devices on a network

In [3]:
bacnet.whois() # Note that this function is called automatically in ReadWriteScript

# The result takes a few milliseconds to be processed... it can be used in the same cell.

[('192.168.56.202', 599),
 ('0xc0a83801dd8f', 262236),
 ('0xc0a83801cfb6', 262237)]

In [172]:
bacnet_2.whois()

[('192.168.56.1:47810', 112233)]

In [4]:
bacnet.whois('0xc0a83801ffc2')

[]

In [5]:
bacnet.whois('192.168.56.202')

[('192.168.56.202', 599)]

In [147]:
bacnet.discover(networks=[1,10,100,1000,2000,10000])  # nie potrzeba znac sieci dla IP; bezposrednie odwolanie po IP

2020-08-24 17:07:07,190 - INFO    | Found those networks : set()
2020-08-24 17:07:07,191 - INFO    | Discovering network 1
2020-08-24 17:07:10,194 - INFO    | Discovering network 10
2020-08-24 17:07:13,200 - INFO    | Discovering network 100
2020-08-24 17:07:16,233 - INFO    | Discovering network 1000
2020-08-24 17:07:19,239 - INFO    | Discovering network 2000
2020-08-24 17:07:22,246 - INFO    | Discovering network 10000


[]

In [6]:
bacnet.devices

2020-10-01 12:18:27,428 - WARNING | APDU Abort Reject Reason : unrecognizedService
2020-10-01 12:18:27,430 - WARNING | Unrecognized service for 192.168.56.202 | 599
2020-10-01 12:18:38,446 - WARNING | APDU Abort Reject Reason : Timeout
2020-10-01 12:18:38,447 - WARNING | No response from controller Timeout
2020-10-01 12:18:38,447 - WARNING | Unrecognized service for 0xc0a83801dd8f | 262236
2020-10-01 12:18:40,453 - WARNING | No response from ('0xc0a83801dd8f', 262236)
2020-10-01 12:18:50,456 - WARNING | APDU Abort Reject Reason : Timeout
2020-10-01 12:18:50,457 - WARNING | No response from controller Timeout
2020-10-01 12:18:50,458 - WARNING | Unrecognized service for 0xc0a83801cfb6 | 262237
2020-10-01 12:18:52,462 - WARNING | No response from ('0xc0a83801cfb6', 262237)


[('Betelgeuse', 'testVendor', '192.168.56.202', 599)]

In [173]:
bacnet_2.devices

2020-08-25 11:08:25,022 - WARNING | APDU Abort Reject Reason : unrecognizedService
2020-08-25 11:08:25,024 - WARNING | Unrecognized service for 192.168.56.1:47810 | 112233


[('TD_1', 'Vendor Name', '192.168.56.1:47810', 112233)]

### Using the bacnet object to read  or write objects from devices

Accessing the list of object of a device can be done using the objectList property


In [174]:
# Let's print the first 10 objects...
bacnet.read('192.168.56.200 device 3056993 objectList')[:10]


[('device', 3056993)]

In [175]:
bacnet.read('192.168.56.1 device 8 objectList')  # symulator YABE nie pozwala przegladac zmiennych (ale bacpypes pozwala)

NoResponseFromController: APDU Abort Reason : Timeout

In [ ]:
bacnet.read('192.168.56.202 device 599 objectList')

In [204]:
obj_list_2 = bacnet.read('192.168.56.1:47810 device 112233 objectList')  # bacpypes samples/handsOnLoab/Samples4_RandomAnalogValueObject.py
print(sorted(obj_list_2, reverse=True))

[('device', 112233)]


In [208]:
fx = BAC0.device('192.168.56.1:47810', 112233, bacnet, segmentation_supported=False, object_list=obj_list_2)#, object_list=sample_obj_list)

2020-08-25 11:28:23,006 - INFO    | Changing device state to DeviceDisconnected'>


InvalidTag: enumerated application tag required

In [ ]:
bacnet.read('192.168.56.202 device 599 objectList') 

In [7]:
sample_obj_list = [('device', 599),
 ('analogValue', 1),
 ('analogValue', 2),
 ('analogValue', 3),
 ('analogValue', 4),
 ('analogValue', 5),
 ('analogValue', 6),
 ('analogValue', 7),
 ('analogValue', 8),
 ('analogValue', 9),
 ('analogValue', 10)]

''' Sample4_RandomAnalogValueObject.py
def main():
    # parse the command line arguments
    args = ConfigArgumentParser(description=__doc__).parse_args()

    if _debug: _log.debug("initialization")
    if _debug: _log.debug("    - args: %r", args)

    # make a device object
    this_device = LocalDeviceObject(
        objectName=args.ini.objectname,
        objectIdentifier=('device', int(args.ini.objectidentifier)),
        maxApduLengthAccepted=int(args.ini.maxapdulengthaccepted),
        segmentationSupported=args.ini.segmentationsupported,
        vendorIdentifier=int(args.ini.vendoridentifier),
        vendorName="testVendor"                                              # <- brakowalo w samplu
        )

    # make a sample application
    this_application = BIPSimpleApplication(this_device, args.ini.address)

    # make some random input objects
    for i in range(1, RANDOM_OBJECT_COUNT+1):
        ravo = RandomAnalogValueObject(
            objectIdentifier=('analogValue', i),
            objectName='Random-%d' % (i,),
            units=64                                                       # <- brakowalo w samplu, z YABE wynika ze 64 to Farenheit
            )
        _log.debug("    - ravo: %r", ravo)
        this_application.add_object(ravo)
'''

fx = BAC0.device('192.168.56.202', 599, bacnet, segmentation_supported=False, object_list=sample_obj_list, poll=0)#, object_list=sample_obj_list)

2020-10-01 12:19:03,268 - INFO    | Changing device state to DeviceDisconnected'>
2020-10-01 12:19:03,472 - INFO    | Changing device state to RPDeviceConnected'>
2020-10-01 12:19:03,492 - INFO    | Device 599:[Betelgeuse] found... building points list
2020-10-01 12:19:03,743 - INFO    | Ready!


In [9]:
fx.points  # zainicjowany wyzej symulacyjny kontroler z bacpypes sample 4

[Betelgeuse/Random-1 : 16.34 degreesFahrenheit,
 Betelgeuse/Random-2 : 91.16 degreesFahrenheit,
 Betelgeuse/Random-3 : 69.76 degreesFahrenheit,
 Betelgeuse/Random-4 : 61.70 degreesFahrenheit,
 Betelgeuse/Random-5 : 9.30 degreesFahrenheit,
 Betelgeuse/Random-6 : 0.76 degreesFahrenheit,
 Betelgeuse/Random-7 : 29.98 degreesFahrenheit,
 Betelgeuse/Random-8 : 55.46 degreesFahrenheit,
 Betelgeuse/Random-9 : 95.59 degreesFahrenheit,
 Betelgeuse/Random-10 : 93.03 degreesFahrenheit]

In [10]:
fx['Random-1']

Betelgeuse/Random-1 : 78.01 degreesFahrenheit

In [164]:
bacnet.readMultiple('192.168.56.202 device 599 Random-2')

ValueError: provide at least one property

In [75]:
bacnet.readMultiple('192.168.56.202 analogValue 1 objectName description')

2020-08-24 16:33:32,245 - WARNING | APDU Abort Reject Reason : unrecognizedService


UnrecognizedService: 

In [ ]:
import time
obj = '2:5 analogValue 41 presentValue'
old_value = bacnet.read(obj)
print('Old Value is : %s' % old_value)
new_value = 120

bacnet.write('%s %s' % (obj,new_value))
print('New value is %s' % bacnet.read(obj))

## Stopping the app

In [169]:
bacnet.disconnect()

2020-08-25 11:07:32,653 - INFO    | Wait while stopping polling
2020-08-25 11:07:32,654 - INFO    | Polling stopped
2020-08-25 11:07:32,742 - INFO    | Device saved to Betelgeuse.db
2020-08-25 11:07:32,742 - INFO    | Changing device state to DeviceFromDB'>
2020-08-25 11:07:32,743 - INFO    | Initializing DB
2020-08-25 11:07:32,765 - INFO    | Device restored from db
2020-08-25 11:07:32,766 - INFO    | You can reconnect to network using : "device.connect(network=bacnet)"
2020-08-25 11:07:32,766 - INFO    | Wait while stopping polling
2020-08-25 11:07:32,767 - INFO    | Polling stopped
2020-08-25 11:07:32,816 - INFO    | Device saved to Betelgeuse.db
2020-08-25 11:07:32,817 - INFO    | Changing device state to DeviceFromDB'>
2020-08-25 11:07:32,818 - INFO    | Initializing DB
2020-08-25 11:07:32,832 - INFO    | Device restored from db
2020-08-25 11:07:32,833 - INFO    | You can reconnect to network using : "device.connect(network=bacnet)"
2020-08-25 11:07:32,834 - INFO    | Wait while s